In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [15]:
source('main.R')

df.train <- data_frame(
    BldgType       = c('A', 'A', 'A', 'B', 'B', 'C', 'C'),
    Quality        = c( 1,   1,   2,   1,   3,   3,   2),
    price          = c(50,   40,  30,  30,   20,   10,   20)
) %>% mutate(id = 1:nrow(.))

df.test <- data_frame(
    BldgType       = c('A', 'A', 'B', 'B', 'C', 'D'),
    Quality        = c( 2,   1,   2,   2,   4,   3),
) %>% mutate(id = 1:nrow(.))


df.train
df.test

BldgType,Quality,price,id
A,1,50,1
A,1,40,2
A,2,30,3
B,1,30,4
B,3,20,5
C,3,10,6
C,2,20,7


BldgType,Quality,id
A,2,1
A,1,2
B,2,3
B,2,4
C,4,5
D,3,6


In [4]:
df.train.tran <-
    df.train %>%
    gather(var.name, var.value, -price, -id) %>%
    group_by(var.name, var.value) %>%
    mutate(avg.price = mean(price)) %>%
    ungroup %>% 
    select(-var.value, -price) %>% 
    spread(var.name, avg.price) %>% 
    arrange(id)

df.train.tran

Z <- 
    df.train.tran %>% 
    inner_join(df.train %>% rename(old.BldgType=BldgType, old.Quality=Quality), by='id')

id,BldgType,Quality
1,40,40
2,40,40
3,40,25
4,25,40
5,25,15
6,15,15
7,15,25


In [5]:
Z %>%
    group_by(old.BldgType) %>%
    mutate(avg.BldgType = mean(price)) %>%
    group_by(old.Quality) %>%
    mutate(avg.Quality = mean(price)) %>%
    mutate(
        avg.BldgType == BldgType,
         avg.Quality == Quality
    )

id,BldgType,Quality,old.BldgType,old.Quality,price,avg.BldgType,avg.Quality,avg.BldgType == BldgType,avg.Quality == Quality
1,40,40,A,1,50,40,40,TRUE,TRUE
2,40,40,A,1,40,40,40,TRUE,TRUE
3,40,25,A,2,30,40,25,TRUE,TRUE
4,25,40,B,1,30,25,40,TRUE,TRUE
5,25,15,B,3,20,25,15,TRUE,TRUE
6,15,15,C,3,10,15,15,TRUE,TRUE
7,15,25,C,2,20,15,25,TRUE,TRUE


In [24]:
source('main.R')

x <- kaggle.house$trans$averagingTransform(df.train, df.test, price, id)

train.tran <- x %>% filter(src == "train") %>% select(-src) %>% arrange(id)
test.tran <- x %>% filter(src == "test") %>% select(-src) %>% arrange(id)

train.tran %>% inner_join(df.train, by='id')
test.tran %>% inner_join(df.test, by='id')

price.x,id,BldgType.x,Quality.x,BldgType.y,Quality.y,price.y
50,1,40,40,A,1,50
40,2,40,40,A,1,40
30,3,40,25,A,2,30
30,4,25,40,B,1,30
20,5,25,15,B,3,20
10,6,15,15,C,3,10
20,7,15,25,C,2,20


price,id,BldgType.x,Quality.x,BldgType.y,Quality.y
NA,1,40.00000,25.00000,A,2
NA,2,40.00000,40.00000,A,1
NA,3,25.00000,25.00000,B,2
NA,4,25.00000,25.00000,B,2
NA,5,15.00000,28.57143,C,4
NA,6,28.57143,15.00000,D,3


In [25]:
source('main.R')

x <- kaggle.house$trans$averagingTransform(df.train, df.test, price, id, diff=T)

train.tran <- x %>% filter(src == "train") %>% select(-src) %>% arrange(id)
test.tran <- x %>% filter(src == "test") %>% select(-src) %>% arrange(id)

train.tran %>% inner_join(df.train, by='id')
test.tran %>% inner_join(df.test, by='id')

price.x,id,BldgType.x,Quality.x,BldgType.y,Quality.y,price.y
50,1,11.428571,11.428571,A,1,50
40,2,11.428571,11.428571,A,1,40
30,3,11.428571,-3.571429,A,2,30
30,4,-3.571429,11.428571,B,1,30
20,5,-3.571429,-13.571429,B,3,20
10,6,-13.571429,-13.571429,C,3,10
20,7,-13.571429,-3.571429,C,2,20


price,id,BldgType.x,Quality.x,BldgType.y,Quality.y
NA,1,11.428571,-3.571429,A,2
NA,2,11.428571,11.428571,A,1
NA,3,-3.571429,-3.571429,B,2
NA,4,-3.571429,-3.571429,B,2
NA,5,-13.571429,0.000000,C,4
NA,6,0.000000,-13.571429,D,3
